## Collecte de données

Création du tableau json

In [6]:
import os 
import json
from PIL import Image

#récupération des données exif d'une image
def get_image_metadata(image_path):
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            format = img.format
            exif_data = img._getexif()

            orientation = 'N/A'
            creation_date = 'N/A'
            camera_model = 'N/A'

            if exif_data:
                orientation = exif_data.get(0x0112) if 0x0112 in exif_data else 'N/A'
                creation_date = exif_data.get(0x0132, 'N/A') if 0x0132 in exif_data else 'N/A'
                camera_model = exif_data.get(0x0110, 'N/A') if 0x0110 in exif_data else 'N/A'
            
            metadata = {
                'filename': os.path.basename(image_path),
                'size': os.path.getsize(image_path),
                'format': format,
                'orientation': orientation,
                'width': width,
                'height': height,
                'creation_date': creation_date,
                'camera_model': camera_model
                # Ajoutez d'autres métadonnées si nécessaire
            } 
            return metadata
    except FileNotFoundError:
        print("Error: Image file not found: {image_path}")
        return {}
    
#parcourt les images du dossier image et crée un tableau avec les infos de ces images
dossier_images = "images"
metadata_images = []

for filename in os.listdir(dossier_images):
    image_path = os.path.join(dossier_images, filename)
    if os.path.isfile(image_path) and os.path.splitext(filename)[-1].lower() in ('.jpg', '.jpeg', '.png'):
        metadata = get_image_metadata(image_path)
        if metadata:
            metadata_images.append(metadata)

#tableau metadata_images dans le fichier json
json_path = "metadata_images.json"
with open(json_path, 'w') as json_file:
    json.dump(metadata_images, json_file, indent=4, default=str)
print("ok")

ok
